<a href="https://colab.research.google.com/github/ZicsX/myAutoScripts/blob/main/pileToParquet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
import datasets

dataset = datasets.load_dataset("monology/pile", split='train', streaming=True)

In [ ]:
import datasets

dataset = datasets.load_dataset("stanford-crfm/DSIR-filtered-pile-50M",split='train', streaming=True)

In [ ]:
print(next(iter(dataset)))

{'contents': 'Alsatian Cheese Tart\n\nFrench Chef Michel Bernard Platz, co-owner with Jose Sanabria of Out of Flower cooking school and specialty catering in Dallas, recommends traditional Alsatian fare. In Alsae, the Yuletide dinner starts with an onion and leek tart, followed by a hearty one-dish creation called Baeckaoffa served with walnut bread. For dessert, try an Alsatian Cheese Tart.\n\nADVERTISEMENT\n\nADVERTISEMENT\n\nADVERTISEMENT\n\nABOUT TEXAS HIGHWAYS\n\nPublished monthly by the Texas Department of Transportation, Texas Highways, the official travel magazine of Texas, encourages travel to and within the Lone Star State and tells the Texas story to readers around the world.search for them."\n\nViktor Troshenkov of the Russian Academy of Sciences told the Tass news agency that the fireball could be part of a prolific meteor shower known as the Leonids, which peaks at this time of year. He said he felt Thursday\'s fireball likely wasn\'t the sole meteorite but others maybe w

In [ ]:
i = 0
for e in dataset['train']:
  if i == 5:
    break
  i+=1
  print(e)

{'text': 'this guy  cracks me up \n---------------------- Forwarded by Drew Fossum/ET&S/Enron on 05/19/2000 \n09:09 AM ---------------------------\n\n05/19/2000 08:29 AM\nLouis Soldano\nLouis Soldano\nLouis Soldano\n05/19/2000 08:29 AM\n05/19/2000 08:29 AM\nTo: Drew Fossum/ET&S/Enron@ENRON\ncc:  \n\nSubject: Re: Emma Caplan  \n\nWell, if she\'s "very bright, personable, and enthusiastic" she certainly \nwon\'t fit in with us.   \n\nI\'ll follow up with mike and shelly on the person we were thiniking of \nsplitting for the summer - that may already be a done deal, if not it may \nmake sense to do something more permanent but we will need to look at \nbudget...\n\ni could probably use some part time help with Bret Reich exiled to Phase IV \nbut i should have a licensed lawyer for that.\n\n\n\n\n   \n\t\n\t\n\tFrom:  Drew Fossum                           05/18/2000 05:50 PM\n\t\n\nTo: Michael Moran/ET&S/Enron@ENRON, Dorothy McCoppin/FGT/Enron@ENRON, Louis \nSoldano/ET&S/Enron@ENRON, Britt

In [ ]:
subset = "Enron Emails"

In [ ]:
from datasets import load_dataset
dataset = load_dataset(
  "ArmelR/the-pile-splitted",
  subset,
  num_proc=8
)

In [ ]:
total_rows = len(dataset['train'])

# Calculate the total number of bytes in the loaded dataset
total_size_bytes = dataset['train'].data.nbytes
total_rows,total_size_bytes

(900246, 1680866499)

In [ ]:
import os
os.makedirs("parquet_files", exist_ok=True)


In [ ]:
# Number of bytes per file (1GB)
size_per_file = 1_000_000_000

# Calculate the number of rows per file
rows_per_file = int((total_rows / total_size_bytes) * size_per_file)
rows_per_file

535584

In [ ]:
from datasets import load_dataset
import pyarrow.parquet as pq
import pyarrow as pa
import os


In [ ]:
# Create a directory to store the Parquet files
os.makedirs("parquet_files", exist_ok=True)

# Calculate the total number of rows
total_rows = len(dataset['train'])

# Calculate the total number of bytes in the loaded dataset
total_size_bytes = dataset['train'].data.nbytes

# Number of bytes per file (1GB)
size_per_file = 1_000_000_000

# Calculate the number of rows per file
rows_per_file = int((total_rows / total_size_bytes) * size_per_file)

# Exporting to multiple Parquet files
start_idx = 0
file_idx = 0
while start_idx < total_rows:
    end_idx = min(start_idx + rows_per_file, total_rows)

    # Select the subset of data for the current file
    subset_data = dataset['train'].select(range(start_idx, end_idx))

    # Convert the dataset to a Table
    subset_table = pa.Table.from_pandas(subset_data.data.to_pandas())

    # Save to Parquet
    pq.write_table(subset_table, f"parquet_files/dataset_{file_idx}.parquet")

    # Update indices
    start_idx = end_idx
    file_idx += 1


In [ ]:
import gc
from datasets import load_dataset, concatenate_datasets
import pyarrow.parquet as pq
import pyarrow as pa
import os
from tqdm import tqdm

data_sources = [
    "NIH ExPorter",
    "PhilPapers",
    "Enron Emails"
    ]

os.makedirs("parquet_files", exist_ok=True)

for subset in tqdm(data_sources, desc="Data Sources"):
    print(f"Processing {subset}...")

    folder_name = subset.replace(" ", "_")

    dataset = load_dataset("ArmelR/the-pile-splitted", subset, num_proc=8)

    concatenated_dataset = concatenate_datasets([dataset['train'], dataset['test']])

    os.makedirs(f"parquet_files/{folder_name}", exist_ok=True)

    total_rows = len(concatenated_dataset)
    total_size_bytes = concatenated_dataset.data.nbytes
    size_per_file = 1_000_000_000
    rows_per_file = int((total_rows / total_size_bytes) * size_per_file)

    start_idx = 0
    file_idx = 0
    pbar = tqdm(total=total_rows, desc=f"Saving {subset}")
    while start_idx < total_rows:
        end_idx = min(start_idx + rows_per_file, total_rows)

        subset_data = concatenated_dataset.select(range(start_idx, end_idx))

        subset_table = pa.Table.from_pandas(subset_data.data.to_pandas())

        pq.write_table(subset_table, f"parquet_files/{folder_name}/dataset_{file_idx}.parquet")

        pbar.update(end_idx - start_idx)

        start_idx = end_idx
        file_idx += 1

    pbar.close()
    print(f"Exported {subset} to {file_idx} Parquet files.")

    del dataset
    del concatenated_dataset
    del subset_data
    del subset_table

    gc.collect()


In [ ]:
dataset = load_dataset(
    "parquet", data_files=["s3://<bucket name>/<data folder>/data-parquet"],
    storage_options=fs.storage_options, streaming=True)

In [ ]:
i = 0
for e in dataset['train']:
  if i == 5:
    break
  i+=1
  print(e)